In [0]:
import requests
from pyspark.sql.functions import col, lit, to_timestamp, current_timestamp, max as spark_max

current_year = 2026
target_table = "f1_processed.races"

try:
    max_round_df = spark.sql(f"SELECT max(round) as max_r FROM {target_table} WHERE race_year = {current_year}")
    current_max_round = max_round_df.collect()[0]['max_r']
    if current_max_round is None: 
        current_max_round = 0
except:
    current_max_round = 0

print(f"🏁 Current Max Round in Delta for {current_year}: {current_max_round}")

url = f"https://api.jolpi.ca/ergast/f1/{current_year}/results.json?limit=100" 
response = requests.get(url)

if response.status_code != 200:
    dbutils.notebook.exit(f"API Error: {response.status_code}")

data = response.json()
total_races = int(data['MRData']['total'])

if total_races == 0:
    print("⏳ Season hasn't started or no results yet.")
    dbutils.notebook.exit("No New Data")

races_list = data['MRData']['RaceTable']['Races']

new_races = [r for r in races_list if int(r['round']) > current_max_round]

if not new_races:
    print("✅ System is up to date. No new races to ingest.")
    dbutils.notebook.exit("No New Data")

print(f"🚀 Found {len(new_races)} new race(s) to ingest!")

clean_data = []
for r in new_races:
    clean_data.append({
        "race_id": int(r['round']), 
        "race_year": int(r['season']),
        "round": int(r['round']),
        "circuit_id": r['Circuit']['circuitId'], 
        "name": r['raceName'],
        "race_timestamp": f"{r['date']} {r['time']}", 
        "data_source": "api_jolpica_weekly"
    })

df_new = spark.createDataFrame(clean_data)

final_df = df_new.select(
    col("race_id"),
    col("race_year"),
    col("round"),
    col("circuit_id"), 
    col("name"),
    to_timestamp(col("race_timestamp"), "yyyy-MM-dd HH:mm:ssZ").alias("race_timestamp"),
    current_timestamp().alias("ingestion_date"),
    col("data_source")
)

final_df.write \
    .mode("append") \
    .format("delta") \
    .saveAsTable(target_table)

print(f"Success: Appended {len(new_races)} new races.")